In [3]:
import datasets

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from datasets import load_dataset
dataset = load_dataset("ai4privacy/pii-masking-300k")


/opt/homebrew/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['source_text', 'target_text', 'privacy_mask', 'span_labels', 'mbert_text_tokens', 'mbert_bio_labels', 'id', 'language', 'set'],
        num_rows: 177677
    })
    validation: Dataset({
        features: ['source_text', 'target_text', 'privacy_mask', 'span_labels', 'mbert_text_tokens', 'mbert_bio_labels', 'id', 'language', 'set'],
        num_rows: 47728
    })
})

In [6]:
#document is rand id
#full_text is simply the text
#tokens is full text split by tokens and put into list
#traling white space is a boolean saying whether there is a space after that token
#labels is 0 if NA or label if it has a label

In [7]:
#up to 29908 are english

In [8]:
document_ids = dataset['train']['id'][:29908]

In [9]:
full_text = dataset['train']['source_text'][:29908]

In [10]:
span_labels = dataset['train']['span_labels'][:29908]

In [11]:
from ast import literal_eval

label_text = []
labels = set()
for i, spans in enumerate(span_labels):
    spans = literal_eval(spans)
    text = full_text[i]
    for span in spans:
        labels.add(span[2])
        text = text[:span[0]] + span[2] + text[span[1]+1:]
    label_text.append(text)

In [12]:
#let's use the spacy english tokenizer
# Construction 2
from spacy.lang.en import English
nlp = English()
# Create a Tokenizer with the default settings for English
# including punctuation rules and exceptions
tokenizer = nlp.tokenizer

In [13]:
tokens = [[i.text for i in tokenizer(i)] for i in full_text]

In [18]:
trailing_whitespaces = tokens = [[bool(i.whitespace_) for i in tokenizer(i)] for i in full_text]

In [15]:
labels = list(labels)

In [16]:
labels

['SEX',
 'CARDISSUER',
 'COUNTRY',
 'GIVENNAME1',
 'DATE',
 'LASTNAME1',
 'STATE',
 'CITY',
 'EMAIL',
 'TITLE',
 'GIVENNAME2',
 'PASSPORT',
 'IP',
 'GEOCOORD',
 'PASS',
 'BUILDING',
 'POSTCODE',
 'USERNAME',
 'LASTNAME3',
 'STREET',
 'LASTNAME2',
 'IDCARD',
 'BOD',
 'DRIVERLICENSE',
 'TEL',
 'SECADDRESS',
 'TIME',
 'SOCIALNUMBER']

In [17]:
label_tokens = [[str(i) for i in tokenizer(i)] for i in label_text]

In [19]:
for label in label_tokens:
    for i,j in enumerate(label):
        label[i] = 'O'
        for l in labels:
            if l in j:
                label[i] = l
                break


In [20]:
data = []

for i in range(len(document_ids)):
    data.append({
        'document': document_ids[i][:-1],
        'full_text': full_text[i],
        'tokens': tokens[i],
        'labels': label_tokens[i],
        'trailing_whitespaces': trailing_whitespaces[i]
    })


In [21]:
import json


with open("train.json", "w") as f:
    f.write(json.dumps(data))